## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBBSSBSS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
584
+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|           coeff_1|            coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|               res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|11205.420962331635| -2366.064654159652|2473.6314804565363|   2499.4|SBBSSBSS| 42.5333|-108.7667|       SNWD|0.11980034930927934|0.08619503239521785|0.040981702483828646|0.8788

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00480443,1965.0,-2366.064654
1,USC00480443,1971.0,-283.725982
2,USC00485105,1986.0,-185.607493
3,USC00482995,1962.0,-4058.941895
4,USC00480443,1984.0,-1655.210172


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,US1WYCR0001,US1WYFM0021,US1WYNT0003,US1WYNT0024,US1WYNT0029,USC00480027,USC00480443,USC00480498,USC00480550,USC00480552,...,USC00488808,USC00489459,USC00489615,USW00024019,USW00024021,USW00024027,USW00024031,USW00024057,USW00024118,USW00024164
year,,,,,,,,,,,,,,,,,,,,,
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,41.904065,200.257955,NaN,3.054171,NaN,657.415000,NaN
1949.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.960902,NaN,NaN,...,NaN,177.212944,NaN,37.008682,552.516025,240.014505,125.788324,NaN,748.410337,320.475487
1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-23.477381,-5.978068
1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,568.135975,NaN
1952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-260.645539,NaN,NaN,-310.382418,NaN,NaN,NaN,-558.766845,-644.948347
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-161.613416,-184.445390,NaN,NaN,NaN,-222.794627,NaN
1954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-47.706539,NaN,...,NaN,-113.360618,NaN,-249.457736,41.070583,-94.019307,NaN,NaN,-111.316992,NaN
1956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.920383,NaN,...,NaN,NaN,NaN,NaN,90.378043,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  640.657058425
RMS removing mean-by-station=  598.857777538
RMS removing mean-by-year   =  494.785769147


In [18]:
for i in range(1, 4):
    i = str(i)
    year_station_table = sqlContext.sql('select station, year, coeff_'+i+' from weather').toPandas().pivot(index='year', columns='station', values='coeff_'+i)
    mean_by_year=np.nanmean(year_station_table,axis=1)
    mean_by_station=np.nanmean(year_station_table,axis=0)
    tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
    tbl_minus_station = year_station_table-mean_by_station

    print '** coeff_'+i+' **'
    print 'total RMS                    = ',RMS(year_station_table)
    print 'RMS removing mean-by-station = ',RMS(tbl_minus_station), ', fraction explained =', (RMS(year_station_table) - RMS(tbl_minus_station)) / RMS(year_station_table) * 100
    print 'RMS removing mean-by-year    = ',RMS(tbl_minus_year), ', fraction explained =', (RMS(year_station_table) - RMS(tbl_minus_year)) / RMS(year_station_table) * 100
    print

** coeff_1 **
total RMS                    =  1848.59752955
RMS removing mean-by-station =  1050.25237611 , fraction explained = 43.1865314478
RMS removing mean-by-year    =  1621.48367705 , fraction explained = 12.2857381809

** coeff_2 **
total RMS                    =  640.657058425
RMS removing mean-by-station =  598.857777538 , fraction explained = 6.52443929831
RMS removing mean-by-year    =  494.785769147 , fraction explained = 22.7690130561

** coeff_3 **
total RMS                    =  619.922538699
RMS removing mean-by-station =  538.19885145 , fraction explained = 13.1828869168
RMS removing mean-by-year    =  491.171563302 , fraction explained = 20.7688811681



In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year     =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations =',RMS(T)

initial RMS= 640.657058425
0 after removing mean by year     = 494.785769147
0 after removing mean by stations = 469.392732
1 after removing mean by year     = 467.426680216
1 after removing mean by stations = 467.088766459
2 after removing mean by year     = 467.002086882
2 after removing mean by stations = 466.973000011
3 after removing mean by year     = 466.961258651
3 after removing mean by stations = 466.955877786
4 after removing mean by year     = 466.953195077
4 after removing mean by stations = 466.951784576
